In [1]:
import firebase_admin
from firebase_admin import db

cred = firebase_admin.credentials.Certificate('/Users/sam/Downloads/flightlog-235e4-firebase-adminsdk-uvrwi-d8510b3147.json')

flightLog = firebase_admin.initialize_app(cred,{'databaseURL':'https://flightlog-235e4.firebaseio.com/'})


In [3]:
import csv
with open("data/ParaglidingLog.csv") as csvfile:
    flights = list(csv.DictReader(csvfile, delimiter=',', quotechar='"'))
    
from datetime import datetime, date

epoch = datetime.utcfromtimestamp(0)

def unix_time_millis(dt):
    return (dt - epoch).total_seconds() * 1000.0

flights = [{}]+flights
for flight in flights:
    if '' in flight.keys():
#         print flight['date'],flight['start_time'],flight['end_time']
#         print flight
        #flight['id']=flight['flight']
        flight['flight']=int(flight['flight'])
        try:
            d = datetime.strptime(flight['date'],'%m/%d/%y')
        except:
            d = datetime.strptime(flight['date'],'%Y-%m-%d')
        flight['date'] = d.isoformat()[:10]
        if flight['start_time']:
            try:
                launchTime = datetime.strptime(flight['date']+' '+flight['start_time'],'%m/%d/%y %I:%M %p')
            except:
                launchTime = datetime.strptime(flight['date']+' '+flight['start_time'],'%Y-%m-%d %I:%M %p')
            flight['launch_time_iso'] = launchTime.isoformat()
#             flight['launch_time_epoch'] = unix_time_millis(launchTime)
        else:
            flight['launch_time_iso'] = None
#             flight['launch_time_epoch'] = None
        if flight['end_time']:
            try:
                landingTime = datetime.strptime(flight['date']+' '+flight['end_time'],'%m/%d/%y %I:%M %p')
            except:
                landingTime = datetime.strptime(flight['date']+' '+flight['end_time'],'%Y-%m-%d %I:%M %p')
            flight['landing_time_iso'] = landingTime.isoformat()
#             flight['landing_time_epoch'] = unix_time_millis(landingTime)
        else:
            flight['landing_time_iso'] = None
#             flight['landing_time_epoch'] = None
        flight['duration_total_minutes']=int(flight['duration_hours'] or 0)*60+int(flight['duration_minutes'] or 0)
        flight['launch_altitude']=int(flight['launch_altitude'] or 0) or None
        flight['landing_altitude']=int(flight['landing_altitude'] or 0) or None
        flight['max_altitude']=int(flight['max_altitude'] or 0) or None
        flight['altitude_gain']=int(flight['altitude_gain'] or 0) or None
        flight['vertical_drop']=int(flight['vertical_drop'] or 0) or None
        flight['xc_miles']=float(flight['xc_miles'] or 0) or None
        flight['total_airtime']=float(flight['total_airtime'])
        
        flight['updated_epoch'] = int(unix_time_millis(datetime.utcnow()))
        
        del flight['']
        del flight['start_time']
        del flight['end_time']
        del flight['duration_hours']
        del flight['duration_minutes']
#         print flight['date'],flight['launch_time_iso'],flight['landing_time_iso'],'\n'
        print flight
    

{'launch_time_iso': None, 'site_name': 'The Point', 'wind_direction': '', 'vertical_drop': None, 'max_altitude': None, 'xc_miles': None, 'comments': 'Training Flights with Santacroce', 'wing_name': '', 'launch_orientation': '', 'updated_epoch': 1520044281724, 'landing_time_iso': None, 'flight': 1, 'launch_name': '', 'altitude_gain': None, 'total_airtime': 0.0, 'date': '2007-04-01', 'wind_speed': '', 'launch_altitude': None, 'landing_altitude': None, 'landing_location': '', 'duration_total_minutes': 0}
{'launch_time_iso': None, 'site_name': 'The Point', 'wind_direction': '', 'vertical_drop': None, 'max_altitude': None, 'xc_miles': None, 'comments': 'Training Flights with Santacroce', 'wing_name': '', 'launch_orientation': '', 'updated_epoch': 1520044281724, 'landing_time_iso': None, 'flight': 2, 'launch_name': '', 'altitude_gain': None, 'total_airtime': 0.0, 'date': '2007-04-01', 'wind_speed': '', 'launch_altitude': None, 'landing_altitude': None, 'landing_location': '', 'duration_total

In [4]:
myFlights = db.reference('/ALdWWkKJGGZZDJSvtdHpgKFxdLt2/flight_log')

In [4]:
#myFlights.set(flights)

In [28]:
#myFlights.set([])

In [5]:
myFlights.set([])
for flight in flights:
   newFlight = myFlights.push()
   newFlight.set(flight)

In [136]:
totalMins=0
lastTotal=0
for flight in flights[21:]:
    totalMins+=flight['duration_total_minutes']
    flight['total_airtime_computed']=totalMins
    
    if abs(flight['duration_total_minutes'] -(flight['total_airtime']*60-lastTotal)) > .5:
        print flight['flight'], flight['duration_total_minutes'], flight['total_airtime'], flight['total_airtime']*60, flight['total_airtime_computed']
    lastTotal = flight['total_airtime']*60
#     if flight['landing_time_epoch']:
#         flight['duration_total_minutes_computed']=int((flight['landing_time_epoch']-flight['launch_time_epoch'])/60000)
#         if flight['duration_total_minutes'] != flight['duration_total_minutes_computed']:
#             print flight['flight'],"check time", flight['launch_time_iso'], flight['landing_time_iso'], flight['duration_total_minutes'], flight['duration_total_minutes_computed'], flight['total_airtime'], flight['total_airtime_computed']/60.
#         if flight['total_airtime'] != flight['total_airtime_computed']/60.:
#             print flight['flight'], flight['total_airtime'], flight['total_airtime']*60, flight['total_airtime_computed']
